In [112]:
import pandas as pd
import math
from time import sleep
from opencage.geocoder import OpenCageGeocode
from clean import *

from emoji import UNICODE_EMOJI

from nmf_helpers import build_text_vectorizer, hand_label_topics, analyze_article
from my_nmf import NMF

with open('loc_data.json','r') as file:
    location_dict = json.load(file)


In [53]:
data = pd.read_csv('data/profile_data.csv')

In [4]:
data = clean(data)
stats(data)

-----------------
CLEANING DATA
...
FOUND 56783 ENTRIES
REMOVED 0 EMPTY ENTRIES
REMOVED 0 DUPLICATED ENTRIES
-----------------
CALCULATING STATS
...
FOUND 56783 ENTRIES
56335 ENTRIES HAVE AGE DATA (99.2%)
19773 ENTRIES HAVE COLLEGE DATA (34.8%)
14456 ENTRIES HAVE CITY DATA (25.5%)
14540 ENTRIES HAVE JOB DATA (25.6%)
10153 ENTRIES HAVE GENDER DATA (17.9%)
55631 ENTRIES HAVE DISTANCE DATA (98.0%)
42256 ENTRIES HAVE DETAIL DATA (74.4%)
15078 ENTRIES HAVE ANTHEM DATA (26.6%)
...
AVERAGE AGE IN DATA SET: 24
NUMBER OF UNIQUE COLLEGES: 7793
NUMBER OF UNIQUE JOBS: 9530
NUMBER OF UNIQUE CITIES: 598


In [29]:
test = data.iloc[2,:].details

In [33]:
bad_chars = ["\n","\r"]

In [36]:
for each in bad_chars:
    test = test.replace(each,"")

In [37]:
test

'Get to know me ☺️ Never been to Disneyland 😅 SC:brix207 Insta: @georgely_ '

In [43]:
def details_stats(details):
    data = {}
    details = details.lower()
    words = details.split(' ')
    data['words'] = len(words)
    
    return data

In [44]:
a = details_stats(test)

In [45]:
a

{'words': 14}

In [51]:
'☺️' in test

True

In [55]:
instagram_tags = ['ig','insta','instagram','📷','📸']
snapchat_tags = ['sc','snap','snapchat','👻']
premium = ['venmo','cashapp','💰']
coronavirus = ['corona','coronavirus','quarantine','covid','covid-19','wfh','work from home','social distancing']

In [152]:
topic = {}
topic['instagram'] = ['ig','insta','instagram','📷','📸','@']
topic['snapchat'] = ['sc','snap','snapchat','👻']
topic['premium snap'] = ['venmo','cashapp','💰']
topic['covid19'] = ['corona','coronavirus','cv','quarantine','covid','covid19','virus','sick','covid-19','wfh','work from home','social distancing','shelter','apocalypse','2020']
topic['cannabis'] = ['weed','420','maryjane','smoke','blaze','marijuana','blunt','bong','joint','high','drug','drugs','cannabis']
topic['dirty'] = ['pussy','dick','penis','vagina','daddy','🍆','👅','💦','fuck','sex']
topic['tv'] = ['netflix','tv','show','hulu','movies','movie','watch','series']
topic['office'] = ['office','jim','pam','dwight','schrute','michael','dunder','mifflin']
topic['himym'] = ['himym']


In [95]:
def word_check(details):
    if type(details)==float:
        return []
    else:
        details = details.lower()
        mentions = []
        for each in topic:
            for word in topic[each]:
                if word in details:
                    mentions.append(each)
                    break
        return mentions
            

In [96]:
word_check(test)

['instagram', 'snapchat']

In [97]:
data['mentions'] = data.details.apply(lambda x: word_check(x))

In [98]:
data['instagram'] = data.mentions.apply(lambda x: True if 'instagram' in x else False)

In [102]:
data['snapchat'] = data.mentions.apply(lambda x: True if 'snapchat' in x else False)

In [101]:
data.instagram.sum()

14923

In [103]:
data.snapchat.sum()

6531

In [111]:
6531/56783

0.11501681841396193

In [110]:
for each in data[[data.snapchat==False]and[data.instagram==False]].details.tolist():
    print(each)

ValueError: Item wrong length 1 instead of 56783.

In [137]:
contents = data[-data.details.isna()]

In [138]:
contents = contents[contents.instagram==False]

In [139]:
contents = contents[contents.snapchat==False]

In [141]:
contents = contents.details

In [142]:
contents

1                                                  A meme 
3        I am a creative person that loves simple, inno...
4                                 Just for fun\r\nI’m 5’2 
7        I’m funny, I can make you laugh I guess:)\r\nA...
8                                        his(panic) 🇲🇽 🇳🇴 
9        Ласка одной львицы лучше, чем гарем дешевых гиен 
11            what are some fun hobbies aside of eating ? 
12       The feeling of rust…. against my salad fingers...
13                                         Mrs. Worldwide 
16       I love singing and adventuring to new countrie...
17       Heyyyy moved to Denmark but I’m from Californi...
22       Отвалились волосы от коронавируса, так что есл...
24       Kinesiology & Nutrition\r\n\r\nYou are respons...
25                                           BHC. Emo kid 
26       Anti-social extrovert. \r\nWhat's the differen...
27                                     bored in the house 
30       im sometimes funny, always tired & bonus point.

In [143]:
vectorizer, vocabulary = build_text_vectorizer(contents,
                             use_tfidf=True,
                             use_stemmer=False,
                             max_features=5000)
X = vectorizer(contents)

In [148]:
for each in vocabulary:
    print(each)

'
'19
0
000
1
10
100
11
12
13
14
15
16
160
165
165cm
167
168
169cm
17
170
170cm
172
173
173cm
174
175
177
178
18
180
19
1999
1st
2
20
2000
2000s
2015
2017
2018
2019
2020
21
22
23
24
25
26
27
28
29
2nd
3
30
31
32
33
35
37
38
3rd
4
4'11
40
420
44
4th
5
5'
5'1
5'10
5'11
5'2
5'3
5'4
5'5
5'6
5'7
5'8
5'9
50
500
54
55
5ft
5th
6
6'
6'0
60
63
64
65
67
68
6ft
7
70
72
73
73cm
74
75
76
78
8
80
80's
80s
8th
9
90
90's
90s
97
98
99
_
aan
abba
aber
ability
able
abogada
abroad
abs
absolute
absolutely
abt
aburrida
aburro
ac
acab
acc
accent
accept
accepting
access
accident
accidentally
according
accordingly
account
accountant
accounting
achieve
acho
across
act
acting
action
actions
active
activist
activities
activity
actor
actress
actriz
actual
actually
acá
add
added
addict
addicted
addiction
administración
administration
admit
adorable
adore
adoro
adrenaline
adult
adults
advance
advantage
adventure
adventurer
adventures
adventuring
adventurous
advertising
advocate
af
affection
affectionate
aficionado
af

engage
engaging
engineer
engineering
england
english
enjoy
enjoying
enjoys
enneagram
enough
enter
entertain
entertained
entertainer
entertaining
entertainment
enthusiast
enthusiastic
entiendo
entire
entj
entp
entre
entrepreneur
então
environment
environmental
episode
equal
equally
equestrian
er
era
eres
es
esfj
eso
esp
español
especial
especially
espero
esportes
espresso
espírito
esquerda
essa
esse
essential
est
esta
estamos
estar
estate
este
esteem
esto
estou
estoy
estudante
estudiando
estudiante
estudio
está
estás
et
etc
eternal
ethically
ethnicity
ett
etter
että
eu
europe
european
even
event
events
eventually
ever
every
everybody
everyday
everyone
everything
everywhere
evil
ex
exactly
excel
excellent
except
exchange
excited
excitement
exciting
excuse
executive
exercise
exercising
exist
existence
existential
exists
exotic
expand
expect
expectations
experience
experiences
experiencing
expert
explain
explore
explorer
exploring
extra
extreme
extremely
extrovert
extroverted
extrovertida


range
rap
rapper
rare
rarely
rat
rate
rather
rating
rato
rave
ravenclaw
raver
raves
ray
rbf
reach
read
reader
reading
reads
ready
real
realist
reality
realize
realized
really
reason
rec
recent
recently
recipes
recommend
recommendations
record
records
recovering
recs
red
reddit
redhead
reference
references
reggae
registered
regret
regrets
regular
regularly
reise
relacionamento
relaciones
related
relation
relations
relationship
relationships
relax
relaxation
relaxing
remember
remind
remotely
renaissance
repeat
reply
republic
requests
required
requirement
requirements
research
researcher
resident
respect
respectable
respectful
respeito
respond
responder
response
responsible
rest
restaurant
restaurants
resting
resto
retired
return
rewatching
rey
reír
ri
rica
rican
rice
rich
rick
rico
rid
ride
ridiculous
riding
rihanna
ring
rings
rio
rip
risada
rising
risk
river
rj
rly
rn
rnb
road
roading
roadtrips
roast
rob
rock
rockabilly
rocks
rocky
role
roll
roller
rollerblading
rollercoasters
rolls
rom

In [146]:
# We'd like to see consistent results, so set the seed.
np.random.seed(12345)

# Find latent topics using our NMF model.
factorizer = NMF(k=3, max_iters=5, alpha=0.5)
W, H = factorizer.fit(X, verbose=True)

iter 0 : reconstruction error: 1027.722776783484
iter 1 : reconstruction error: 293.6433050073501
iter 2 : reconstruction error: 158.95489272828357
iter 3 : reconstruction error: 146.14762289434898
iter 4 : reconstruction error: 144.84333081877742
FINAL reconstruction error: 144.17823831488775 



In [147]:
# Label topics and analyze a few NYT articles.
hand_labels = hand_label_topics(H, vocabulary)

topic 0
--> 5 looking i'm la hi someone 1 want let serious get find trying 7 2 meet send take know 8
please label this topic: looking

topic 1
--> love like fun good music make hey know let friends new laugh go brazilian life traveling always adventures travel english
please label this topic: travel

topic 2
--> quarantine bored inst time talk friends made new people need lover want tinder name girl trade pic written world like
please label this topic: quarantine



In [130]:
hand_labels

['bored',
 'social media',
 'quarantine',
 'tinder',
 'lonely',
 'friends',
 'tiktok']

In [129]:
rand_articles = np.random.choice(range(len(W)), 15)

for i in rand_articles:
    analyze_article(i, contents, web_urls, W, hand_labels)

TypeError: 'int' object is not subscriptable